In [2]:
# lo_polygon = ox.geocode_to_gdf('R176095', by_osmid=True).to_crs(epsg=32636)
# spb_polygon = ox.geocode_to_gdf('R337422', by_osmid=True).to_crs(epsg=32636).buffer(3000)
# city = lo_polygon.union(spb_polygon).to_crs(epsg=4326) #  get lo polygon
# polygon = gpd.GeoDataFrame([{'name': 'Ленинградская область', 'geometry': city[0]}], crs="EPSG:4326")

In [3]:
# gg= he.convert_list_attr_to_str(g.graph)
# for node,e,data in gg.edges(data=True):
#     if 'geometry' in data:
#         data['geometry']=str(data['geometry'])
# nx.write_graphml(gg, f"tula.graphml")

In [1]:
import sys
sys.path.append('/Users/polina/Desktop/github/transport_frames')
from transport_frames.graphbuilder.graph import Graph 
from transport_frames.frame_grader.advanced_grade import AdvancedGrader
from transport_frames.framebuilder.frame import Frame
import transport_frames.utils.helper_funcs as he
from transport_frames.indicators.indicator_terr import indicator_territory
from transport_frames.indicators.indicator_area import indicator_area
from transport_frames.indicators.utils import assign_services_names_to_nodes, availability_matrix
import osmnx as ox
import geopandas as gpd
import networkx as nx
from shapely import Polygon, MultiPolygon
import momepy
import shapely




regions = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/lo_gdfs/regions_of_russia.geojson')
regions = regions[regions['ISO3166-2']!='RU-CHU']
regions = regions[regions['geometry'].apply(lambda x: isinstance(x, (Polygon, MultiPolygon)))]

polygon = ox.geocode_to_gdf('R81993',by_osmid=True)

cities = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/Тульская_область_region_points.geojson')
cities = cities[cities.is_admin_centre_district==1].reset_index()
cities = cities[['name','geometry']]


/Users/polina/Desktop/github/transport_frames/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## create graph and frame

In [5]:
local_crs = 32637

In [6]:
# g = Graph.from_polygon(polygon, crs=local_crs)
tula = he.convert_geometry_from_wkt(nx.read_graphml('tula.graphml'))
tula = he.convert_list_attr_from_str(tula)
g =  Graph(tula,crs=local_crs)

2024-09-04 17:22:40.224 | INFO     | transport_frames.new_modules.graphbuilder.graph:_prepare_attrs:135 - Preparing the graph...
2024-09-04 17:22:50.052 | INFO     | transport_frames.new_modules.graphbuilder.graph:_prepare_attrs:173 - Graph is ready!


In [7]:
f = Frame(g.graph, regions,polygon,cities)

## grade terr (by frame)

In [8]:
poly = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/settlement.geojson').reset_index()

In [9]:
n_grade = f.grade_territory(poly, include_priority=True)


## grade terr (advanced)

In [10]:
inter = nx.read_graphml('/Users/polina/Desktop/github/transport_frames/preprocessed/Тульская_область/Тульская_область_inter.graphml')
inter = he.prepare_graph(inter)

settlements = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/settlement.geojson')

b_stops = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/bus_stop.geojson')
r_stops = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/railway_station.geojson')
aero = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/aerodrome.geojson')
points = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/Тульская_область_region_points.geojson')

In [11]:
grader = AdvancedGrader(32637)
cri = grader.get_criteria(g.graph,n_grade,points,settlements,inter,r_stops,b_stops,aero,aero)

2024-09-04 17:24:23.038 | INFO     | dongraphio.dongraphio:get_adjacency_matrix:84 - Creating adjacency matrix based on provided graph...
2024-09-04 17:24:27.475 | DEBUG    | dongraphio.base_models.matrix_logic:get_adjacency_matrix:52 - Calculating distances ...
2024-09-04 17:27:25.017 | INFO     | dongraphio.dongraphio:get_adjacency_matrix:93 - Adjacency matrix done!
/Users/polina/Desktop/github/transport_frames/.venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3577: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
2024-09-04 17:27:25.285 | INFO     | dongraphio.dongraphio:get_adjacency_matrix:84 - Creating adjacency matrix based on provided graph...
2024-09-04 17:27:38.418 | DEBUG    | dongraphio.base_models.matrix_logic:get_adjacency_matrix:52 - Calculating distances ...


In [ ]:
cri

,geometry,grade,weight,weight_r_stops,weight_b_stops,weight_ferry,weight_aero,car_access_quartile,public_access_quartile,car_grade,public_transport_grade,overall_assessment
0,"POLYGON ((822739.390 5891144.015, 822856.784 5...",5.0,0.7,0.35,0.35,0.0,0.0,4,4,2,1.4,1.70
1,"POLYGON ((770575.741 6018486.567, 772068.280 6...",5.0,1.0,0.35,0.35,0.2,0.1,1,1,5,5.0,5.00
2,"POLYGON ((749855.647 6043718.447, 750071.385 6...",5.0,0.7,0.35,0.35,0.0,0.0,3,3,3,2.1,2.55
3,"POLYGON ((833357.642 6005750.703, 833750.462 6...",5.0,0.7,0.35,0.35,0.0,0.0,3,2,3,2.8,2.90
4,"POLYGON ((815882.425 6032567.814, 816000.612 6...",5.0,1.0,0.35,0.35,0.2,0.1,3,2,3,4.0,3.50
...,...,...,...,...,...,...,...,...,...,...,...,...
79,"POLYGON ((755657.835 6010239.121, 755739.325 6...",3.0,1.0,0.35,0.35,0.2,0.1,2,3,3,2.0,2.50
80,"POLYGON ((814205.213 6004629.644, 814212.553 6...",3.5,0.7,0.35,0.35,0.0,0.0,1,1,4,3.5,3.75
81,"POLYGON ((739451.181 5959327.073, 739565.593 5...",3.0,0.7,0.35,0.35,0.0,0.0,3,4,1,0.7,0.85
82,"POLYGON ((710232.263 5998683.611, 710421.182 5...",5.0,1.0,0.35,0.35,0.2,0.1,4,4,2,2.0,2.00


## indicators terr

In [ ]:
inter = nx.read_graphml('/Users/polina/Desktop/github/transport_frames/preprocessed/Тульская_область/Тульская_область_inter.graphml')
inter = he.prepare_graph(inter)
poly = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/settlement.geojson').reset_index()
settlements = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/settlement.geojson')

b_stops = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/bus_stop.geojson')
r_stops = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/railway_station.geojson')
aero = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/aerodrome.geojson')
points = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/Тульская_область_region_points.geojson')
regions_gdf = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/district.geojson')

In [ ]:
custom_territory=poly.iloc[[5]]
custom_territory_center = custom_territory.geometry.representative_point()
custom_territory_center = gpd.GeoDataFrame([{'geometry': custom_territory_center.iloc[0]}], crs=custom_territory_center.crs).to_crs(32637)


In [ ]:
PLACEHOLDER = gpd.GeoDataFrame(geometry=[]) # если нет данных о сервисе

railway_stations = gpd.read_file(f'/Users/polina/Desktop/github/transport_frames/data/Тульская_область/railway_station.geojson')
railway_roads = gpd.read_file(f'/Users/polina/Desktop/github/transport_frames/data/Тульская_область/railway.geojson')
bus_stops = gpd.read_file(f'/Users/polina/Desktop/github/transport_frames/data/Тульская_область/bus_stop.geojson')
bus_routes = gpd.read_file(f'/Users/polina/Desktop/github/transport_frames/preprocessed/Тульская_область/Тульская_область_bus_routes.geojson')
fuel_stations = gpd.read_file(f'/Users/polina/Desktop/github/transport_frames/data/Тульская_область/Копия fuel.geojson')
ferry_terminal = PLACEHOLDER
local_aerodrome = gpd.read_file(f'/Users/polina/Desktop/github/transport_frames/data/Тульская_область/aerodrome.geojson')
local_aerodrome['geometry'] = shapely.centroid(local_aerodrome['geometry']).set_crs(local_aerodrome.crs)
# international_aerodrome = gpd.read_file(f'/Users/sashamorozov/Documents/НЦКР платформа ЦУ/Показатели по 9 регионам/data/data/{region_name}/{region_name}_international_aerodrome.geojson')
# либо:
international_aerodrome = local_aerodrome[local_aerodrome['aerodrome:type']=='international']
international_aerodrome['geometry'] = shapely.centroid(international_aerodrome['geometry']).set_crs(international_aerodrome.crs)

# Природные объекты
water = gpd.read_file(f'/Users/polina/Desktop/github/transport_frames/data/Тульская_область/Копия water.geojson')
nature_reserve = gpd.read_file(f'/Users/polina/Desktop/github/transport_frames/data/Тульская_область/Копия nature_reserve.geojson')


In [ ]:
services = {
    'railway_stops' : railway_stations,
    'fuel' : fuel_stations,
    'ports' : ferry_terminal,
    'local_aero' : local_aerodrome,
    'international_aero' : international_aerodrome,
    'oopt' : nature_reserve,
    'water_objects' : water,
    'train_paths' : railway_roads,
    'bus_stops' : bus_stops,
    'capital': ox.geocode_to_gdf('N34389350',by_osmid=True),
    'points': points,
    'bus_routes' : bus_routes,
    'region_centers': cities,
    'neud_center': custom_territory_center

}

In [ ]:
G = assign_services_names_to_nodes(services,momepy.nx_to_gdf(g.graph)[0],g.graph,crs = 32637)

railway_stops
fuel
no ports (((((((
local_aero
no international_aero (((((((
no oopt (((((((
no water_objects (((((((
no train_paths (((((((
bus_stops
capital
points
bus_routes
region_centers
neud_center


In [ ]:
t = indicator_territory(G,poly.iloc[[5]],3857,regions_gdf, points, cities, services, *momepy.nx_to_gdf(g.graph))
t

100%|██████████| 1/1 [00:00<00:00,  7.81it/s]


nearest_region_centers
nearest_settlement_centers


,name,to_region_admin_center,to_reg1,fuel_stations_accessibility,train_stops_accessibility,local_aero_accessibility,international_aero_accessibility,ports_accessibility,water_objects_accessibility,oopt_accessibility,...,number_of_international_aero,number_of_bus_stops,number_of_water_objects,number_of_oopt,density,train_path_length,number_of_bus_routes,to_nearest_settlement_center,to_nearest_mo_center,geometry
29800,None,94.138,2.132,0,0,100.492,None,None,0.002,2.716,...,0,12,11,0,4444.598,19.209,64,1757.859,1757.859,"POLYGON ((36.08847 53.81940, 36.09122 53.81675..."


In [ ]:
inter = he.prepare_graph(inter)

In [ ]:
b = indicator_area(G, inter, services, [settlements], 32637)

100%|██████████| 2986/2986 [04:16<00:00, 11.65it/s]


epsg:32637
{'crs': 'epsg:32637', 'approach': 'primal', 'graph_type': 'car graph'}
new version
epsg:32637
{'node_default': {}, 'edge_default': {}, 'crs': 'epsg:32637', 'graph_type': 'intermodal graph', 'walk speed': 66.67, 'car speed': 283.33, 'subway speed': 200.0, 'tram speed': 250.0, 'trolleybus speed': 200.0, 'bus speed': 283.33}
new version


/Users/polina/Desktop/github/transport_frames/transport_frames/new_modules/indicators/utils.py:176: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  result = polygons_gdf.merge(length_sums, how='left', left_on='index', right_on='index').fillna(0)
/Users/polina/Desktop/github/transport_frames/transport_frames/new_modules/indicators/utils.py:173: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  result = polygons_gdf.merge(length_sums, how='left', left_on='index', right_on='index').fillna(0)
/Users/polina/Desktop/github/transport_frames/transport_frames/new_modul

In [ ]:
b[0]

,index,name,status,layer,geometry,fuel_stations_accessibility,train_stops_accessibility,local_aero_accessibility,international_aero_accessibility,ports_accessibility,...,number_of_fuel_stations,number_of_train_stops,number_of_local_aero,number_of_international_aero,number_of_ports,number_of_bus_stops,reg1_length,reg2_length,reg3_length,number_of_bus_routes
0,0,Манаенское,сельское поселение,Арсеньевский муниципальный район,"POLYGON ((36.27046 53.67571, 36.27343 53.67505...",29.949,43.470,84.710,None,None,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,112.522,158.473,0.000
1,1,Астаповское,сельское поселение,Арсеньевский муниципальный район,"POLYGON ((36.47061 53.63576, 36.48947 53.63114...",15.841,36.302,63.376,None,None,...,1.000,0.000,0.000,0.000,0.000,8.000,0.000,181.931,303.156,0.000
2,2,Арсеньево,городское поселение,Арсеньевский муниципальный район,"POLYGON ((36.63005 53.72754, 36.63174 53.72702...",1.450,32.167,56.096,None,None,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,8.566,48.936,0.000
3,3,Левобережное,сельское поселение,Белевский муниципальный район,"POLYGON ((35.89856 53.85018, 35.90015 53.84679...",14.235,16.179,110.678,None,None,...,0.000,0.000,0.000,0.000,0.000,33.000,98.842,119.191,566.839,0.000
4,4,Правобережное,сельское поселение,Белевский муниципальный район,"POLYGON ((36.14550 53.80533, 36.14561 53.80468...",13.650,17.654,100.875,None,None,...,1.000,0.000,0.000,0.000,0.000,3.000,0.000,66.226,284.629,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,79,Яснополянское,сельское поселение,Щекинский муниципальный район,"POLYGON ((37.17494 54.04491, 37.17570 54.04310...",6.130,7.852,60.310,None,None,...,1.000,0.000,0.000,0.000,0.000,20.000,18.837,89.941,186.581,0.000
80,80,Ясногорск,городское поселение,Ясногорский муниципальный район,"POLYGON ((37.66036 54.50418, 37.67147 54.49780...",1.031,0.632,29.522,None,None,...,2.000,1.000,0.000,0.000,0.000,40.000,0.000,22.769,123.129,3.000
81,81,Теляковское,сельское поселение,Ясногорский муниципальный район,"POLYGON ((37.60388 54.56328, 37.60784 54.55777...",16.434,23.488,40.752,None,None,...,0.000,0.000,0.000,0.000,0.000,21.000,0.000,114.506,336.889,0.000
82,82,Ревякинское,сельское поселение,Ясногорский муниципальный район,"POLYGON ((37.47214 54.48182, 37.47222 54.48006...",12.704,8.431,24.174,None,None,...,1.000,1.000,0.000,0.000,0.000,47.000,16.718,133.655,379.176,3.000
